In [1]:
import numpy as np

In [2]:
import import_ipynb
from fakeENS import fakeENS
from spectralEM import spectralEM
from scipy.stats import entropy

importing Jupyter notebook from fakeENS.ipynb
importing Jupyter notebook from spectralEM.ipynb


In [3]:
def str_arr(arr):
    return " ".join("%.2f"%x for x in arr)
#--------------------------------------------
def str_mat(mat):
    return "\n".join(str_arr(arr) for arr in mat) 

In [4]:
num_classifier = 5
num_class = 3

ph_const = 0.1
ph_true = np.random.rand(num_class); 
ph_true = (ph_const + ph_true)/sum(ph_const + ph_true)

siz = int(1e4)
real_labels = np.random.choice(range(num_class), p = ph_true, size = siz)
ph_emp = np.bincount(real_labels)/ float(len(real_labels))

E = fakeENS(num_classifier, num_class, real_labels)
C_real = E.getConfMat()

In [5]:
S = spectralEM(num_classifier, num_class, 
               maxiter = 100, num_init = 200, thres_const = 5e-4, disp = False)

print 'ph_true:%s \nph_emp:%s'%(str_arr(ph_true), str_arr(ph_emp))

group = S.group 
groupConfMat = {i: np.mean([C_real[j] for j in range(num_classifier) if group[i][j]], axis = 0)  for i in range(3)}
kappa = np.min([np.min([[groupConfMat[j][l,l] - groupConfMat[j][l,c] for c in range(num_class) if c !=l] 
                      for l in range(num_class)]) for j in range(3)])


barD = np.min([[np.mean([entropy(C_real[i][l,:], C_real[j][l,:]) 
                         for i in range(num_classifier)]) 
                for c in range(num_class) if c !=l] 
               for l in range(num_class)])

for i in range(3): print "Group %d:\n%s"%(i,str_mat(groupConfMat[i]));
print '\nkappa:%.3f, barD:%.3f'%(kappa, barD)

print 'Initial eigenvectors:\n%s'%str_mat(S.v_arr)

ph_true:0.42 0.25 0.33 
ph_emp:0.42 0.25 0.33
Group 0:
0.61 0.17 0.22
0.19 0.57 0.24
0.13 0.25 0.62
Group 1:
0.54 0.05 0.41
0.28 0.59 0.13
0.39 0.08 0.53
Group 2:
0.71 0.02 0.27
0.23 0.47 0.30
0.18 0.12 0.69

kappa:0.128, barD:0.106
Initial eigenvectors:
0.29 0.79 0.57
0.81 0.17 -0.53
-0.51 -0.58 0.63


In [12]:
E.reshuffle()
S.reset()

update_period = 500
est_period = 2*update_period
new_data = []
disp = True
errp_best = 100


for i in range(siz):
    schedule = (i+1)*np.ones(num_classifier).astype(int)
    labels = E.classify(schedule) - 1
    #print 'index:', i, 'real label:',real_labels[i], 'classifier labels:', labels
    new_data += [labels.astype(int)]
    if np.mod(i, update_period) == 0 and i>0: 
        S.update(new_data); 
        #-----------------------
        if np.mod(i, est_period) == 0 and i >0: 
            S.updateParamsSpectral('check')
        #-----------------------
        S.updateParamsEM(new_data, 10);
        new_data = []
    #------------------
        errp = np.linalg.norm(S.ph_est_avg - ph_true, 1)
        err = [np.linalg.norm(C_real[k]- S.conf_mat[k], 1) for k in range(num_classifier)]
        if errp_best > errp:
            ph_est_best = S.ph_est_avg.copy()
            conf_mat_best = S.conf_mat.copy()
            errp_best = errp
        #-----------------------------------------
        if disp:
            print '---------------------------'
            print 'Num samples:', S.num_data
            print 'Total L1 error in ph true: %.3f'%errp
            print 'Total L1 error in Conf Matrices: %.3f'%np.sum(err)
            print '---------------------------'
#------------------------------------------------------

---------------------------
Num samples: 501
Total L1 error in ph true: 0.164
Total L1 error in Conf Matrices: 2.513
---------------------------
---------------------------
Num samples: 1001
Total L1 error in ph true: 0.369
Total L1 error in Conf Matrices: 4.036
---------------------------
---------------------------
Num samples: 1501
Total L1 error in ph true: 0.369
Total L1 error in Conf Matrices: 4.416
---------------------------
---------------------------
Num samples: 2001
Total L1 error in ph true: 0.297
Total L1 error in Conf Matrices: 4.038
---------------------------
---------------------------
Num samples: 2501
Total L1 error in ph true: 0.297
Total L1 error in Conf Matrices: 3.969
---------------------------
---------------------------
Num samples: 3001
Total L1 error in ph true: 0.703
Total L1 error in Conf Matrices: 2.599
---------------------------
---------------------------
Num samples: 3501
Total L1 error in ph true: 0.703
Total L1 error in Conf Matrices: 2.358
-------

In [13]:
print 'ph_est:%s \nph_true:%s \nph_emp:%s'%(str_arr(S.ph_est_avg), str_arr(ph_true), str_arr(ph_emp))

for k in range(num_classifier):
    print '---------------------------'
    print 'Classifier:%d'%k
    print 'Confusion Matrix Final Estimate'
    print np.round(S.conf_mat[k],3)
    print 'Confusion Matrix Real'
    print np.round(C_real[k],3)
print '---------------------------'

ph_est:0.30 0.47 0.22 
ph_true:0.42 0.25 0.33 
ph_emp:0.42 0.25 0.33
---------------------------
Classifier:0
Confusion Matrix Final Estimate
[[0.382 0.113 0.505]
 [0.221 0.396 0.383]
 [0.281 0.302 0.417]]
Confusion Matrix Real
[[0.655 0.    0.344]
 [0.266 0.444 0.289]
 [0.047 0.149 0.804]]
---------------------------
Classifier:1
Confusion Matrix Final Estimate
[[0.381 0.218 0.4  ]
 [0.    1.    0.   ]
 [0.422 0.265 0.313]]
Confusion Matrix Real
[[0.593 0.311 0.096]
 [0.34  0.524 0.137]
 [0.045 0.191 0.764]]
---------------------------
Classifier:2
Confusion Matrix Final Estimate
[[0.566 0.    0.434]
 [0.396 0.476 0.129]
 [0.    1.    0.   ]]
Confusion Matrix Real
[[0.538 0.052 0.41 ]
 [0.282 0.59  0.127]
 [0.393 0.076 0.531]]
---------------------------
Classifier:3
Confusion Matrix Final Estimate
[[0.405 0.16  0.435]
 [0.    1.    0.   ]
 [0.169 0.409 0.422]]
Confusion Matrix Real
[[0.63  0.024 0.346]
 [0.037 0.612 0.352]
 [0.222 0.304 0.474]]
---------------------------
Classifier:

In [11]:
print 'ph_est:%s \nph_true:%s \nph_emp:%s'%(str_arr(ph_est_best), str_arr(ph_true), str_arr(ph_emp))
for k in range(num_classifier):
    print '---------------------------'
    print 'Classifier:%d'%k
    print 'Confusion Matrix Best Estimate'
    print np.round(conf_mat_best[k],3)
    print 'Confusion Matrix Real'
    print np.round(C_real[k],3)
print '---------------------------'

ph_est:0.47 0.24 0.29 
ph_true:0.42 0.25 0.33 
ph_emp:0.42 0.25 0.33
---------------------------
Classifier:0
Confusion Matrix Best Estimate
[[0.407 0.126 0.467]
 [0.328 0.391 0.281]
 [0.317 0.165 0.517]]
Confusion Matrix Real
[[0.655 0.    0.344]
 [0.266 0.444 0.289]
 [0.047 0.149 0.804]]
---------------------------
Classifier:1
Confusion Matrix Best Estimate
[[0.411 0.318 0.271]
 [0.    1.    0.   ]
 [0.383 0.237 0.38 ]]
Confusion Matrix Real
[[0.593 0.311 0.096]
 [0.34  0.524 0.137]
 [0.045 0.191 0.764]]
---------------------------
Classifier:2
Confusion Matrix Best Estimate
[[1.    0.    0.   ]
 [0.    0.733 0.267]
 [0.    0.301 0.699]]
Confusion Matrix Real
[[0.538 0.052 0.41 ]
 [0.282 0.59  0.127]
 [0.393 0.076 0.531]]
---------------------------
Classifier:3
Confusion Matrix Best Estimate
[[0.416 0.224 0.36 ]
 [0.    1.    0.   ]
 [0.341 0.237 0.422]]
Confusion Matrix Real
[[0.63  0.024 0.346]
 [0.037 0.612 0.352]
 [0.222 0.304 0.474]]
---------------------------
Classifier:4
Co